### Baseline модель для определения именованных сущностей по кейсу от Rutube.
Поскольку нам нужно распознать нестандартные NER, можно воспользоваться помощью языковых моделей, в данном случае - Bert.
Данные вы уже получили  - это разметка, сделанная на Толоке, она не идеальна, но это часть практической задачи, с которой можно столкнуться в реальности.

Небольшое введение в NER https://habr.com/ru/companies/contentai/articles/449514/

In [ ]:
# считаем данные
import pandas as pd
data = pd.read_csv("/content/drive/MyDrive/ner_DP/ner_data_train.csv")

In [ ]:
data

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"{""label"":""локация""\,""offset"":26\,""length"":6\,""..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"{""label"":""организация""\,""offset"":196\,""length""..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"{""label"":""название проекта""\,""offset"":12\,""len..."
3,<НАЗВАНИЕ:> Довоенная немецкая кирха в Калинин...,"{""label"":""не найдено""\,""offset"":162\,""length"":..."
4,"<НАЗВАНИЕ:> ""Спартаку"" помогли судьи? Локомоти...","{""label"":""команда""\,""offset"":13\,""length"":8\,""..."
...,...,...
6417,<НАЗВАНИЕ:> ЗАПРЕЩЕННЫЙ УДАР! Полный бой Вадим...,"{""label"":""персона""\,""offset"":41\,""length"":12\,..."
6418,<НАЗВАНИЕ:> Как создаются медали: оптические и...,"{""label"":""локация""\,""offset"":88\,""length"":10\,..."
6419,"<НАЗВАНИЕ:> Ремонт ломовоза камаз, монтаж гидр...","{""label"":""бренд""\,""offset"":28\,""length"":5\,""se..."
6420,<НАЗВАНИЕ:> Голодные игры: Баллада о змеях и п...,"{""label"":""Дата""\,""offset"":78\,""length"":4\,""seg..."


In [ ]:
# данные спарсены с Толоки, поэтому могут иметь проблемы с символами и их нужно избежать,
# удалить лишние '\' например, преобразовать из str в список dict-ов
import json
df = data.copy()
df['entities'] = df['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df['entities'] = df['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

In [ ]:
df.head(3)

,video_info,entities
0,<НАЗВАНИЕ:> Агент 117: Из Африки с любовью — Р...,"[{'label': 'локация', 'offset': 26, 'length': ..."
1,<НАЗВАНИЕ:> Коленвал Инфинити Ку икс 56= 5.6 V...,"[{'label': 'организация', 'offset': 196, 'leng..."
2,<НАЗВАНИЕ:> ВЫЗОВ ДЕМОНА = Вызвал Серого Челов...,"[{'label': 'название проекта', 'offset': 12, '..."


#### Оригинал туториала на медицинских данных можно посмотреть тут https://gist.github.com/avidale/cacf235aebeaaf4c578389e1146c3c57

In [ ]:
!pip install razdel

In [ ]:
# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

In [ ]:
ner_train[0]

In [ ]:
print(extract_labels(df.iloc[0]))

{'tokens': ['<', 'НАЗВАНИЕ', ':', '>', 'Агент', '117', ':', 'Из', 'Африки', 'с', 'любовью', '—', 'Русский', 'тизер', '=', 'трейлер', '(', '2021', ')', '<', 'ОПИСАНИЕ', ':', '>', 'Лучший', 'Telegram', 'канал', 'о', 'кино', '<', 'LINK', '>', 'Сотрудничество', '<', 'LINK', '>', 'Дата', 'выхода', '26', 'августа', '2021', 'Оригинальное', 'название', ':', 'OSS', '117', ':', 'Alerte', 'rouge', 'en', 'Afrique', 'noire', 'Страна', ':', 'Франция', 'Режиссер', ':', 'Николя', 'Бедос', 'Жанр', ':', 'боевик', ',', 'комедия', 'В', 'главных', 'ролях', ':', 'Жан', 'Дюжарден', ',', 'Пьер', 'Нинэ', ',', 'Мелоди', 'Каста', ',', 'Наташа', 'Линдинжер', ',', 'Владимир', 'Иорданов', ',', 'Фату', 'Н', '’', 'Диайе', ',', 'Пол', 'Уайт', 'Мир', 'изменился', '.', 'Он', 'нет', '.', 'Судьба', 'заносит', 'легендарного', 'Агента', '117', 'в', 'Африку', ',', 'где', 'горячее', 'пустыни', 'только', 'женщины', '.', 'Вооруженный', 'неиссякаемой', 'уверенностью', 'в', 'себе', 'и', 'убийственной', 'харизмой', ',', 'он', 'мож

### Обработаем датасет и разобьем на трейн и тест

In [ ]:
from sklearn.model_selection import train_test_split
ner_data = [extract_labels(item) for i, item in df.iterrows()]
ner_train, ner_test = train_test_split(ner_data, test_size=0.2, random_state=1)

NameError: ignored

In [ ]:
import pandas as pd
pd.options.display.max_colwidth = 300
pd.DataFrame(ner_train).sample(10)

,tokens,tags
2995,"[<, НАЗВАНИЕ, :, >, ДРИФТ, КОРЧ, ИЗ, КАМАЗА, !, ОПЕРСКОЙ, КАМАЗ, С, МАКСИМАЛКОЙ, 300, КМ, /, Ч, В, GTA, :, КРМП, (, RADMIR, RP, CRMP, ), <, ОПИСАНИЕ, :, >, Играю, в, GTA, :, КРМП, ., Криминальная, Россия, на, Radmir, ', e, ., Создал, самый, лютый, камаз, на, сервере, в, опер, дрифт, стиле, !, На...","[O, O, O, O, O, O, O, B-модель, O, O, B-модель, O, O, O, O, O, O, O, B-видеоигра, O, B-видеоигра, O, B-видеоигра, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1632,"[<, НАЗВАНИЕ, :, >, Сначала, гоночка, ,, затем, елочка, !, Как, праздновать, Новый, год, от, команды, RUMOS, RACING, !, <, ОПИСАНИЕ, :, >, Сначала, гоночка, ,, затем, елочка, !, Как, праздновать, Новый, год, от, команды, RUMOS, RACING, !, Как, пилоты, отметили, окончание, сезона, и, наступление,...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-команда, I-команда, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-команда, I-команда, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
1647,"[<, НАЗВАНИЕ, :, >, Воспитанные, волками, \, ВЫШЛИ, 2, СЕРИИ, ВТОРОГО, СЕЗОНА, \, трейлер, <, ОПИСАНИЕ, :, >, <, LINK, >, Год, производства, 2020, (, 2, сезона, ), Страна, США, Жанр, фантастика, ,, триллер, ,, драма, 2145, год, ., Человечество, захлебнулось, в, кровопролитной, войне, между, веру...","[O, O, O, O, O, O, O, O, O, O, B-сезон, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]"
2547,"[<, НАЗВАНИЕ, :, >, ФИЛЬМ, ЧЕРНОБЫЛЬ, 2021, ИЛИ, КОНЕЦ, КАРЬЕРЫ, ДАНИЛЫ, КОЗЛОВСКОГО, ?, <, ОПИСАНИЕ, :, >, Всем, привет, !, В, этом, видео, я, расскажу, вам, ,, как, я, и, Вкусаня, сходили, в, кинотеатр, на, фильм, Чернобыль, 2021, ., Авария, на, Чернобыльской, АЭС, ставит, под, угрозу, мечту, ...","[O, O, O, O, B-название проекта, I-название проекта, B-Дата, O, O, O, B-персона, I-персона, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-персона, O, O, O, O, O, B-название проекта, I-название проекта, O, O, O, B-организация, I-организация, O, O, O, O, B-персона, O, O, O, O, O, O, O, O..."
335,"[<, НАЗВАНИЕ, :, >, Teenage, Mutant, Ninja, Turtles, полное, прохождение, на, русском, языке, Dendy, Денди, NES, Nintendo, Famicom, <, ОПИСАНИЕ, :, >, СОГЛАСЕН, НА, ВЗАИМНУЮ, ПОДПИСКУ, ПИШИ, В, КОМЕНТЕ, Прохождение, со, взломом, (, хаком, ), игры, на, здоровье, ., Очень, веселой, интересной, и, ...","[O, O, O, O, B-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, I-видеоигра, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, B-видеоигра, I-видеоигра, I-видеоигра, I-видеоигра,..."
2562,"[<, НАЗВАНИЕ, :, >, КВН, Радио, Свобода, =, 2016, Высшая, лига, Первая, 1/8, Приветствие, <, ОПИСАНИЕ, :, >, КВН, 2016, Высшая, лига, Первая, 1/8, YouTube, <, LINK, >, Официальный, сайт, КВН, <, LINK, >, /, Vkontakte, <, LINK, >, Приобрести, билет, на, игры, Высшей, Лиги, КВН, <, LINK, >, #, KVN...","[O, O, O, O, B-организация, B-организация, I-организация, O, B-Дата, B-название проекта, I-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, B-организация, B-Дата, B-название проекта, I-название проекта, I-название проекта, I-название проекта, O, O, O, O, ..."
1663,"[<, НАЗВАНИЕ, :, >, YV, =, 2022, Шорт, =, лист, –, "", Мой, Магнитогорск, "", –, Алексей, Рогалин, (, Магнитогорск, ), <, ОПИСАНИЕ, :, >, IV, Международный, фестиваль, социальных, роликов, "", Твой, <, AT, >, "", Организаторы, :, Культурный, центр, "", Век, "", г, ., Магнитогорска, ,, Объединение, гор...","[O, O, O, O, O, O, B-Дата, O, O, O, O, O, O, O, O, O, B-персона, I-персона, O, B-локация, O, O, O, O, O, O, O, O, O, O,

#### Посмотрим на получившиеся теги
Подробнее почитать про BIO теги можно тут https://datascience.stackexchange.com/questions/63399/what-is-bio-tags-for-creating-custom-ner-named-entity-recognization

In [ ]:
label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
label_list

['O',
 'B-Дата',
 'B-бренд',
 'B-вид спорта',
 'B-видеоигра',
 'B-команда',
 'B-лига',
 'B-локация',
 'B-модель',
 'B-название проекта',
 'B-организация',
 'B-персона',
 'B-сезон',
 'B-серия',
 'I-Дата',
 'I-бренд',
 'I-вид спорта',
 'I-видеоигра',
 'I-команда',
 'I-лига',
 'I-локация',
 'I-модель',
 'I-название проекта',
 'I-организация',
 'I-персона',
 'I-сезон',
 'I-серия']

In [ ]:
from datasets import Dataset, DatasetDict

In [ ]:
ner_data = DatasetDict({
    'train': Dataset.from_pandas(pd.DataFrame(ner_train)),
    'test': Dataset.from_pandas(pd.DataFrame(ner_test))
})
ner_data

NameError: ignored

### Запустим модель RuBert-tiny - классический Bert, поверх которого навешен слой классификации токенов.

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

model_checkpoint = "cointegrated/rubert-tiny"
batch_size = 16

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, device='cpu')

In [ ]:
example = ner_train[5]
print(example["tokens"])

NameError: ignored

In [ ]:
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print(tokens)

['[CLS]', '<', 'Н', '##АЗ', '##В', '##АН', '##И', '##Е', ':', '>', 'Московский', 'между', '##народный', 'фестиваль', 'мира', '=', '89', 'и', 'С', '##тас', 'На', '##мин', '.', '«', 'Главный', 'день', '»', '<', 'О', '##П', '##И', '##С', '##АН', '##И', '##Е', ':', '>', '12', 'и', '13', 'июня', '1989', 'года', 'на', 'Центр', '##альном', 'стадион', '##е', 'имени', 'В', '.', 'И', '.', 'Ленина', 'состоялся', 'первый', 'между', '##народный', 'рок', 'фестиваль', '.', 'К', '##он', '##цер', '##т', 'со', '##брал', 'свыше', '100', 'тысяч', 'зрителей', '.', 'Советская', 'м', '##оло', '##де', '##ж', '##ь', 'тогда', 'у', '##вид', '##ела', 'и', 'у', '##сл', '##ыш', '##ала', 'признан', '##ных', 'мир', '##овых', 'к', '##уми', '##ров', ':', 'Sc', '##orp', '##ions', ',', 'Bon', 'Jovi', ',', 'Oz', '##zy', 'Os', '##bour', '##ne', ',', 'Ci', '##nder', '##ella', ',', 'Mot', '##ley', 'C', '##rue', '.', 'При', '##езд', 'запад', '##ных', 'зв', '##езд', 'каз', '##ался', 'о', '##жив', '##шей', 'ф', '##ант', '##аст'

In [ ]:
len(example["tags"]), len(tokenized_input["input_ids"])

(170, 274)

In [ ]:
print(tokenized_input.word_ids())

[None, 0, 1, 1, 1, 1, 1, 1, 2, 3, 4, 5, 5, 6, 7, 8, 9, 10, 11, 11, 12, 12, 13, 14, 15, 16, 17, 18, 19, 19, 19, 19, 19, 19, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 29, 30, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 39, 40, 41, 42, 43, 43, 43, 43, 44, 44, 45, 46, 47, 48, 49, 50, 51, 51, 51, 51, 51, 52, 53, 53, 53, 54, 55, 55, 55, 55, 56, 56, 57, 57, 58, 58, 58, 59, 60, 60, 60, 61, 62, 63, 64, 65, 65, 66, 66, 66, 67, 68, 68, 68, 69, 70, 70, 71, 71, 72, 73, 73, 74, 74, 75, 75, 76, 76, 77, 77, 77, 78, 78, 78, 78, 79, 80, 80, 80, 81, 82, 83, 84, 85, 86, 87, 88, 88, 89, 90, 90, 90, 91, 92, 92, 93, 93, 94, 94, 95, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 104, 105, 106, 106, 106, 107, 108, 109, 110, 110, 110, 111, 112, 113, 114, 115, 116, 117, 117, 117, 118, 118, 118, 118, 119, 120, 121, 121, 122, 123, 124, 124, 125, 125, 126, 126, 127, 127, 127, 128, 128, 129, 130, 130, 130, 130, 131, 132, 132, 133, 133, 134, 135, 136, 137, 137, 137, 138, 139, 139, 139, 140, 141, 142, 143, 144, 144, 144, 

In [ ]:
word_ids = tokenized_input.word_ids()
aligned_labels = [-100 if i is None else example["tags"][i] for i in word_ids]
print(len(aligned_labels), len(tokenized_input["input_ids"]))

274 274


#### У Bert свой собсвенный токенайзер, который разбивает слова на мелкие токены, поэтому нам нужно корректно сопоставить токены и соответсвующие им неры.

In [ ]:
def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples['tags']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(ner_data['train'][1:2])

{'input_ids': [[2, 32, 293, 16218, 8117, 20795, 8759, 14386, 30, 34, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 32, 294, 3932, 8759, 3330, 20795, 8759, 14386, 30, 34, 294, 18147, 5972, 2386, 1928, 733, 22970, 613, 17565, 29484, 15479, 12122, 18398, 13334, 1, 16992, 12449, 19314, 9147, 705, 19062, 1211, 2179, 15349, 603, 18, 282, 3200, 860, 20727, 12029, 4375, 29463, 2013, 2262, 18, 294, 15735, 29137, 320, 18029, 18398, 24952, 753, 705, 11316, 4307, 329, 22067, 12427, 283, 4297, 292, 29172, 3330, 1928, 705, 13790, 294, 22797, 1768, 10203, 761, 13951, 3243, 2535, 2641, 18, 3]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [ ]:
tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Map:   0%|          | 0/5137 [00:00<?, ? examples/s]

Map:   0%|          | 0/1285 [00:00<?, ? examples/s]

#### Сохраняем словарик соотвествия тега и его индекса внутри модели

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))
model.config.id2label = dict(enumerate(label_list))
model.config.label2id = {v: k for k, v in model.config.id2label.items()}

Some weights of the model checkpoint at cointegrated/rubert-tiny were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny and are newly

In [ ]:
# Специальный объект для удобного формирования батчей
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

### В качестве метрик возьмем precision, recall, accuracy, для этого воспользуемся специализированной под Ner задачу библиотеку seqeval

In [ ]:
metric = load_metric("seqeval")

/var/folders/pw/qs4ys1kd68sbr86_46f_tgz80000gp/T/ipykernel_20329/152412463.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
example = ner_train[4]
labels = example['tags']
metric.compute(predictions=[labels], references=[labels])

{'бренд': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'локация': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [ ]:
# что мы видим без дообучения модели
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels, zero_division=0)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.evaluate()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


RuntimeError: Placeholder storage has not been allocated on MPS device!

In [ ]:
pip install torch torchvision torchaudio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
  Obtaining dependency information for torchaudio from https://files.pythonhosted.org/packages/9e/e2/812b1a2a5ec34fe7c0a5cf11b4e62709172fd71783cbc8b7875e3051e8a5/torchaudio-2.1.0-cp39-cp39-macosx_11_0_arm64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.9 kB/s eta 0:00:00m eta 0:00:010:00:01m

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import logging
from transformers.trainer import logger as noisy_logger
noisy_logger.setLevel(logging.WARNING)

In [ ]:
# Для дообучения берта можно эксперементировать с заморозкой/разморозкой разных слоев, здесь мы оставим все слои размороженными
# Для быстроты обучения можно заморозить всю бертовую часть, кроме классификатора, но тогда качесвто будет похуже
for param in model.parameters():
    param.requires_grad = True

In [ ]:
args = TrainingArguments(
    "ner",
    evaluation_strategy = "epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    weight_decay=0.01,
    save_strategy='no',
    report_to='none',
)

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
# Посчитаем метрики на отложенном датасете

predictions, labels, _ = trainer.predict(tokenized_datasets["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric.compute(predictions=true_predictions, references=true_labels)
results

In [ ]:
from sklearn.metrics import confusion_matrix
import pandas as pd

In [ ]:
cm = pd.DataFrame(
    confusion_matrix(sum(true_labels, []), sum(true_predictions, []), labels=label_list),
    index=label_list,
    columns=label_list
)
cm

In [ ]:
model.save_pretrained('ner_bert.bin')
tokenizer.save_pretrained('ner_bert.bin')

### Посмотрим на результаты

In [3]:
!pip install transformers
!pip install razdel
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 97.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 87.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
import pandas as pd
import json
import torch

data1 = pd.read_csv("/content/drive/MyDrive/ner_DP/ner_data_test.csv")

df1 = data1.copy()
df1['entities'] = df1['entities'].apply(lambda l: l.replace('\,', ',')if isinstance(l, str) else l)
df1['entities'] = df1['entities'].apply(lambda l: l.replace('\\\\', '\\')if isinstance(l, str) else l)
df1['entities'] = df1['entities'].apply(lambda l: '[' + l + ']'if isinstance(l, str) else l)
df1['entities'] = df1['entities'].apply(lambda l: json.loads(l)if isinstance(l, str) else l)

# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = item['entities']
    if isinstance(labels, dict):
        labels = [labels]
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

# ner_data = [extract_labels(item) for i, item in df1.iterrows()]


In [ ]:
ner_data[0]

In [ ]:
# import torch
# from transformers import pipeline
# from transformers import AutoModelForTokenClassification
# from transformers import AutoTokenizer

# def predict_ner(text, tokenizer, model, pipe, verbose=True):
#     tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
#     tokens = {k: v.to(model.device) for k, v in tokens.items()}

#     with torch.no_grad():
#         pred = model(**tokens)
#     # print(pred.logits.shape)
#     indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
#     token_text = tokenizer.convert_ids_to_tokens(tokens['input_ids'][0])
#     labels = []
#     for t, idx in zip(token_text, indices):
#         if '##' not in t:
#             labels.append(label_list[idx])
#         if verbose:
#             print(f'{t:15s} {label_list[idx]:10s}')
#     return text, pipe(text), labels

# def tokenize_and_align_labels(examples, label_all_tokens=True):
    # tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    # labels = []
    # for i, label in enumerate(examples['tags']):
    #     word_ids = tokenized_inputs.word_ids(batch_index=i)
    #     previous_word_idx = None
    #     label_ids = []
    #     for word_idx in word_ids:
    #         # Special tokens have a word id that is None. We set the label to -100 so they are automatically
    #         # ignored in the loss function.
    #         if word_idx is None:
    #             label_ids.append(-100)
    #         # We set the label for the first token of each word.
    #         elif word_idx != previous_word_idx:
    #             label_ids.append(label[word_idx])
    #         # For the other tokens in a word, we set the label to either the current label or -100, depending on
    #         # the label_all_tokens flag.
    #         else:
    #             label_ids.append(label[word_idx] if label_all_tokens else -100)
    #         previous_word_idx = word_idx

    #     label_ids = [label_list.index(idx) if isinstance(idx, str) else idx for idx in label_ids]

    #     labels.append(label_ids)

    # tokenized_inputs["labels"] = labels
    # return tokenized_inputs

In [50]:
from transformers import pipeline
from transformers import AutoModelForTokenClassification
from transformers import AutoTokenizer

model_check = '/content/drive/MyDrive/ner_DP/distilbert/ner_bert'

model = AutoModelForTokenClassification.from_pretrained(model_check) # num_labels=len(label_list)
tokenizer = AutoTokenizer.from_pretrained(model_check, device='cpu')
pipe = pipeline(model=model, tokenizer=tokenizer, task='ner', aggregation_strategy='average', device='cpu')


In [ ]:
# from datasets import Dataset, DatasetDict

# ner_data = DatasetDict({
#     'data': Dataset.from_pandas(pd.DataFrame(ner_data))
# })

# tokenized_datasets = ner_data.map(tokenize_and_align_labels, batched=True)

In [ ]:
ner_data[0]

### Тестового датасета у вас пока нет, по которому будет считаться метрика на лидерборде, но прогоним для примера через нашу отложенную выборку, чтобы понять формат выходных данных.
ВАЖНО: в тестовом датасете у вас будет тест в том же формате, что он был в трейне 'video_info', в финальном сабмишене эту колонку и индексы менять нельзя, нужно будет только заполнить колонку 'entities_prediction'

In [44]:
import torch
from transformers import pipeline
import re
from tqdm import tqdm

# Теперь из наших данных нам нужно извлечь для каждого слова (токена) его тег (label) из разметки, чтобы потом предать в модель классификации токенов
from razdel import tokenize

def extract_labels(item):

    # воспользуемся удобным токенайзером из библиотеки razdel,
    # она помимо разбиения на слова, сохраняет важные для нас числа - начало и конец слова в токенах

    raw_toks = list(tokenize(item['video_info']))
    words = [tok.text for tok in raw_toks]
    # присвоим для начала каждому слову тег 'О' - тег, означающий отсутствие NER-а
    word_labels = ['O'] * len(raw_toks)
    char2word = [None] * len(item['video_info'])
    # так как NER можем состаять из нескольких слов, то нам нужно сохранить эту инфорцию
    for i, word in enumerate(raw_toks):
        char2word[word.start:word.stop] = [i] * len(word.text)

    labels = None
    if labels is not None:
        for e in labels:
            if e['label'] != 'не найдено':
                e_words = sorted({idx for idx in char2word[e['offset']:e['offset']+e['length']] if idx is not None})
                if e_words:
                    word_labels[e_words[0]] = 'B-' + e['label']
                    for idx in e_words[1:]:
                        word_labels[idx] = 'I-' + e['label']
                else:
                    continue
            else:
                continue
        return {'tokens': words, 'tags': word_labels}
    else: return {'tokens': words, 'tags': word_labels}

def predict_ner(text, tokenizer, model, pipe, verbose=True):

    dismissed_token = re.compile(r'\xad+|\u200b+')
    text = [re.sub(dismissed_token, '[UNK]', tok) for tok in text]

    # print(text)

    tokens = tokenizer(text, truncation=True, is_split_into_words=True, return_tensors='pt')
    words = tokens.words()

    tokens = {k: v.to(model.device) for k, v in tokens.items()}

    with torch.no_grad():
        pred = model(**tokens)

    indices = pred.logits.argmax(dim=-1)[0].cpu().numpy()
    # print('indices', indices)
    labels = []
    prev=words[1] # это всегда ноль - первое слово
    # print(prev)
    labels = [label_list[indices[1]]]
    # print(labels)
    for word, tag in zip(words[1:-1], indices[1:-1]):
        if word != prev:
            labels.append(label_list[tag])
            prev=word
    return labels


submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
submission['entities_prediction'] = submission['entities_prediction'].astype('object')

def sample_submission(df, tokenizer, model, pipe, submission):

    for i, elem in tqdm(df.iterrows()):
        tokens = extract_labels(elem)
        labels = predict_ner(tokens['tokens'], tokenizer, model, pipe, verbose=False)
        if len(tokens['tokens'])!= len(labels):
            print(tokens['tokens'])
        submission.loc[i, 'video_info'] = elem['video_info']

        submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
    return submission

In [ ]:
res.to_csv('/content/drive/MyDrive/ner_DP/result_total_2_1.csv', index=False)

In [ ]:
# from tqdm.notebook import tqdm

# submission = pd.DataFrame(columns=[['video_info', 'entities_prediction']])
# submission['entities_prediction'] = submission['entities_prediction'].astype('object')

# def sample_submission(data, tokenizer, model, pipe, submission):
#     i = 0
#     for elem, text in tqdm(zip(data, data1['video_info'])):
#         # print(elem)
#         # print(text)
#         _, _, labels = predict_ner(elem['tokens'], tokenizer, model, pipe, verbose=False)
#         submission.loc[i, 'video_info'] = text

#         submission.loc[i, 'entities_prediction'] = [[label] for label in labels]
#         i += 1
#     return submission
'''
['<', 'НАЗВАНИЕ', ':', '>', 'Масложор', 'после', 'удаления', 'катализатора', 'и'
['<', 'НАЗВАНИЕ', ':', '>', '33', '💎с', 'Сэмюэлем', '.', 'Я', 'ОХОЧУСЬ', 'НА', 'ТЕБЯ',
['<', 'НАЗВАНИЕ', ':', '>', 'ДОГАДКИ', ',', 'ГИПОТИЗЫ', 'И', 'СТРАШН
['<', 'НАЗВАНИЕ', ':', '>', 'ВЛАД', 'ОСОЗНАЁТ', 'ОШИ
<', 'НАЗВАНИЕ', ':', '>', 'ТАНЦЫ', 'С', 'СОБАЧКОЙ',
['<', 'НАЗВАНИЕ', ':', '>', 'Вариаторный', 'впуск', 'или', 'впуск',
['<', 'НАЗВАНИЕ', ':', '>', 'Проблема', 'опускания', 'люльки', 'на', 'автовышке', 'Manotti', '

In [74]:
l1 = [len(x[0]) for x in result['entities_prediction'].values.tolist()]

In [126]:
l2 = [len(x.split(',')) for x in list(pd.read_csv('/content/submission.csv')['entities_prediction'].iloc[1145])]
l1 = [len(x.split(',')) for x in list(pd.read_csv('/content/drive/MyDrive/ner_DP/result_total_2.csv')['entities_prediction'].iloc[1145])]

In [127]:
# for c, (i, j) in enumerate(zip(l1, l2)):
#   if i != j:
#     print(i, j, c)
len(l1), len(l2)

(940, 970)

In [121]:
len(eval(res.iloc[1145]['entities_prediction']))

194

In [108]:
print(eval(pd.read_csv('/content/submission.csv').iloc[1145]['entities_prediction']))

['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [124]:
res = pd.read_csv('/content/drive/MyDrive/ner_DP/result_total_2.csv')
res.iloc[1145]['entities_prediction'] = str(['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O'])

In [125]:
res.to_csv('/content/drive/MyDrive/ner_DP/result_total_2_1.csv', index=False)

In [119]:
res.iloc[1145]['entities_prediction'])

TypeError: ignored

In [ ]:
['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [85]:
pd.read_csv('/content/submission.csv')

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","['O', 'O', 'O', 'O', 'B-название проекта', 'I-..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"['O', 'O', 'O', 'O', 'B-название проекта', 'I-..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"['O', 'O', 'O', 'O', 'B-персона', 'I-персона',..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"['O', 'O', 'O', 'O', 'B-персона', 'B-персона',..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"['O', 'O', 'O', 'O', 'B-персона', 'I-персона',..."


In [52]:
label_list = sorted({label for item in ner_data for label in item['tags']})
# label_list = sorted({label for item in ner_train for label in item['tags']})
if 'O' in label_list:
    label_list.remove('O')
    label_list = ['O'] + label_list
# ner_data = ner_data[:2]
result = sample_submission(data1[['video_info']], tokenizer, model, pipe, submission)

0it [00:00, ?it/s]/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:360: FutureWarning: `BatchEncoding.words()` property is deprecated and should be replaced with the identical, but more self-explanatory `BatchEncoding.word_ids()` property.
  warnings.warn(
1146it [07:25,  2.30it/s]

['<', 'НАЗВАНИЕ', ':', '>', 'НА', 'КОГО', 'ПОТРАТИТЬ', '20', 'АЛМАЗОВ', 'В', '3', 'СЕРИИ', '_', 'УЛУЧШЕНИЕ', 'ОТНОШЕНИЙ', '_', 'СЕКРЕТ', 'НЕБЕС', '_', 'КЛУБ', 'РОМАНТИКИ', '<', 'ОПИСАНИЕ', ':', '>', '#', 'romanceclub', '#', 'клубромантики', '#', 'прохождение', '#', 'игра', '#', 'новелла', '#', 'визуальнаяновелла', '#', 'гайд', '#', 'аудиокнига', '#', 'Озвучка', '#', 'ДИЛ', '#', 'Идеал', '#', 'ЯОНТ', '#', 'ЯТ', '#', 'РЛ', '#', 'ЛИ', '#', 'ПВ', '#', 'КЗ', '30', 'Д', '#', 'СБиП', '#', 'ПСИ', '#', 'ЦиОТ', '#', 'Теодора', '#', 'Аркакнум', '#', 'ХГ', '#', 'ПТЛ', '#', 'ГЛ', '#', 'ДЖС', '#', 'ЛСЗ', '#', 'СН', '#', 'ВП', '#', 'ТС', '#', 'ПвТ', '#', 'МГИ', '#', 'ВРС', '#', 'ПВ', '#', 'КЗТ', '#', 'Озвучка_КР', '#', 'ДИЛ_КР', '#', 'Идеал_КР', '#', 'ЯОНТ_КР', '#', 'ЯТ_КР', '#', 'РЛ_КР', '#', 'ЛИ_КР', '#', 'ПВ_КР', '#', 'КЗ', '30', 'Д_КР', '#', 'СБиП_КР', '#', 'ПСИ_КР', '#', 'ЦиОТ_КР', '#', 'Теодора_КР', '#', 'Аркакнум_КР', '#', 'ХГ_КР', '#', 'ПТЛ_КР', '#', 'ГЛ_КР', '#', 'ДЖС_КР', '#', 'ЛСЗ_КР', '#'

1606it [10:25,  2.57it/s]


In [53]:
result

,video_info,entities_prediction
0,"<НАЗВАНИЕ:> БОЕЦ БЕЗ ПРАВИЛ, ТРЕЙЛЕР на русско...","[O, O, O, O, B-название проекта, I-название пр..."
1,<НАЗВАНИЕ:> ШОК! КАК ЖЕ ЭТОМУ БОМЖУ ВЕЗЕТ В Br...,"[O, O, O, O, O, O, O, O, O, O, O, O, B-видеоиг..."
2,<НАЗВАНИЕ:> ДРЕВНИЕ РОБОТЫ NATASHA TALON И LEO...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
3,<НАЗВАНИЕ:> ЖЕНА ПУТЕШЕСТВЕННИКА ВО ВРЕМЕНИ = ...,"[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
4,<НАЗВАНИЕ:> Кинозал ДК приглашает на мультфиль...,"[O, O, O, O, O, O, O, O, O, O, B-название прое..."
...,...,...
1601,"<НАЗВАНИЕ:> Милейшие детеныши животных, которы...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ..."
1602,<НАЗВАНИЕ:> Диана Шурыгина набросилась на зрит...,"[O, O, O, O, B-персона, I-персона, O, O, O, O,..."
1603,<НАЗВАНИЕ:> Губернатор Евгений Куйвашев и непо...,"[O, O, O, O, O, B-персона, I-персона, O, O, O,..."
1604,<НАЗВАНИЕ:> Александр Горелов = о вакцинации о...,"[O, O, O, O, B-персона, I-персона, O, O, O, O,..."


In [54]:
result.to_csv('/content/drive/MyDrive/ner_DP/result_total_2.csv', index=False)

In [ ]:
len(ner_test)